# Automobile Seats Manufacturing Optimization

Weekly Output (per worker) by Production Line and Seat Style (thousands of units)


Production Line| 0|1|2
---------------|------- |-------|-------
0|20|30|40
1|50|35|45

Production Line Parameters

Production Line|Worker Wages| Organization Cost| Max Workers
---------------|------------|------------------|------------
0|400|1000|30
1|600|2000|30

Minimum Demand

style|Demand
---------------|------------
style_0|120
style_1|150
style_2|200

In [ ]:
# ! pip install -q pyomo
# ! pip install cplex -q
# !apt-get install -y -qq coinor-cbc

In [ ]:
import pandas as pd


# Load the Excel file with data and parameters we will need
xls = pd.ExcelFile('/content/drive/MyDrive/Colab Notebooks/Optimization with Python/Linear Programming/Automobile Seats - Cost Optimization 2 dimensions/automobile_seat_data.xlsx')

# Read each sheet
worker_production = xls.parse('worker_production')
production_line_params = xls.parse('production line params')
min_demand = xls.parse('min_demand')

In [ ]:
worker_production.head()

,production_line,0,1,2
0,0,20,30,40
1,1,50,35,45


In [ ]:
production_line_params.head()

,production_line,salary,prod_cost,max_workers
0,0,400,1000,30
1,1,600,2000,30


In [ ]:
min_demand.head()

,style,min_demand
0,style_0,120
1,style_1,150
2,style_2,200


In [ ]:
# Extract data

n_lines = len(worker_production.production_line)
n_styles = len(worker_production.columns) - 1

print(n_lines,n_styles)

2 3


In [ ]:
param_dict = {}

for i in range(len(production_line_params.columns)):
  param_dict[production_line_params.columns[i]] = production_line_params.iloc[:,i].tolist()

worker_production_dict = {}

for j in range(len(worker_production.columns)):
  worker_production_dict[worker_production.columns[j]] = worker_production.iloc[:,j].tolist()

min_demand_dict = dict(zip(min_demand.index, min_demand.iloc[:,1].tolist()))

print(param_dict,'\n',worker_production_dict,'\n', min_demand_dict)

{'production_line': [0, 1], 'salary': [400, 600], 'prod_cost': [1000, 2000], 'max_workers': [30, 30]} 
 {'production_line': [0, 1], 0: [20, 50], 1: [30, 35], 2: [40, 45]} 
 {0: 120, 1: 150, 2: 200}


Set up the model and variables

In [ ]:
from pyomo.environ import *

model = ConcreteModel()

model.i = Set(initialize=set(worker_production_dict['production_line']))
model.j = Set(initialize=worker_production_dict.keys()-{'production_line'})
i = model.i
j = model.j

model.salary = Param(i, initialize=dict(zip(param_dict['production_line'], param_dict['salary'])))
model.prod_cost = Param(i, initialize=dict(zip(param_dict['production_line'], param_dict['prod_cost'])))
model.max_workers = Param(i, initialize=dict(zip(param_dict['production_line'], param_dict['max_workers'])))
model.min_demand = Param(j, initialize=min_demand_dict)

salary = model.salary
prod_cost = model.prod_cost
max_workers = model.max_workers
min_demand = model.min_demand


In [ ]:
# Assuming worker production is per worker per seat style per production line
model.worker_production = Param(i, j, initialize={(i, j): worker_production_dict[j][i] for i in model.i for j in model.j})
worker_production = model.worker_production

# x is number of workers assigned to each production line and seat style
model.x = Var(i, domain=NonNegativeIntegers)
# b is a binary variable indicating if a production line is used
model.b = Var(i, domain=Binary)



x = model.x
b = model.b

Set up the model objective:

<center> $ \min(Total Cost) = \min \left[\sum_i PC_{i} \times b_{i} +\sum_{i} Salary_{i}\times x_i\right]$

In [ ]:
def objective_function(model):
  # return sum(model.prod_cost[i] * model.b[i] for i in i) + sum(model.salary[i] * model.x[i] for i in i)
  return sum(model.prod_cost[i] * model.b[i] + model.salary[i] * model.x[i] for i in i)

model.obj = Objective(expr=objective_function, sense=minimize)

Set up the constraints to satisfy the demand for each seat style $j$:

<center> $\sum S_{i,j} \times x_i \ge D_j\space\space\space \forall j$ </center>








In [ ]:
# Set up the constraints to satisfy the demand for each seat style j:

def min_demand_constraint(model, j):
  # Total production of style_ across all lines i
  return sum(worker_production[i, j] * x[i] for i in i) >= min_demand[j]

model.demand_constraint = Constraint(j, rule=min_demand_constraint)


Set up constraint for max workers on each production line $i$:

<center> $x_i \le 30 \times b_i \space\space\space \forall i$</center>

<center> $x_i \ge 0 \space\space\space \forall i$ and $x$ is integer</center>

In [ ]:
def max_workers_constraint_rule(model, i):
  # Total workers on line i
  return x[i] <= max_workers[i]

# Corrected indexing for max_workers_constraint
model.max_workers_constraint = Constraint(i, rule=max_workers_constraint_rule)

# x_i,j >= 0 for all i, j (already handled by domain=NonNegativeIntegers)
# b_i is binary (already handled by domain=Binary)

In [ ]:
solver = SolverFactory('cbc')
results = solver.solve(model)

In [ ]:
print(results)


Problem: 
- Name: unknown
  Lower bound: 2200.0
  Upper bound: 2200.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Number of binary variables: 2
  Number of integer variables: 4
  Number of nonzeros: 2
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.02
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.06445193290710449
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [ ]:
print('Objective Function = ', model.obj())
for i in i:
  print('Number of workers employed in Production Line',i, 'is =', x[i]())

Objective Function =  2200.0
Number of workers employed in Production Line 0 is = 4.0
Number of workers employed in Production Line 1 is = 1.0
